# Applied Data Science Capstone - week 3 part 1

### Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Exploring and cluster the neighborhoods in Toronto

In [72]:
# import libraries for Dataframe
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import packages for web scrapping: beautifulsoup & requests
import requests
import json # library to handle requests JSON files
from bs4 import BeautifulSoup
from urllib.request import urlopen
import codecs

#import map rendering libraries
import folium 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


##### 2. Import url

Obtaining the data that is in the table of postal codes and to transform it into a pandas dataframe

In [131]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = pd.read_html(url)
toronto_data = toronto_data[0] #Select first dataframe of the page

##### 3. Create the data frame

* Create columns

In [132]:
toronto_data.columns = ['Postal Code','Borough','Neighborhood'] #Rename Columns
toronto_data = toronto_data.drop([0]) #Remove extraneous header from df

* Remove 'Not Assigned' from boroughs, replace 'Not assigned' Neighborhoods with Borough value and merge neighborhoods existing in more than one postal code area

In [133]:
#remove 'Not Assigned' from Boroughs
toronto_data['Borough'].replace('Not assigned',np.nan,inplace = True)
toronto_data.dropna(inplace=True)

#Replace 'Not assigned' Neighborhoods with Borough value
toronto_data.reset_index(drop = True,inplace = True)  
for x in toronto_data.index:  
    if (toronto_data.loc[x]['Neighborhood'] == 'Not assigned'):  
        toronto_data.loc[x]['Neighborhood'] = toronto_data.loc[x]['Borough']

In [134]:
droplist = []  
for x in range(1,len(toronto_data.index.unique()-1)):  
    if (toronto_data.loc[x-1]['Postal Code'] == toronto_data.loc[x]['Postal Code']):  
        toronto_data.loc[x]['Neighborhood'] = toronto_data.loc[x]['Neighborhood'] + (', '+toronto_data.loc[x-1]['Neighborhood'])  
        droplist.append(x-1)  
toronto_data.drop(droplist, inplace=True)

* print the number of rows in the dataframe

In [135]:
toronto_data.sort_values(by='Postal Code',inplace=True)
toronto_data.reset_index(drop = True,inplace = True)
print('Number of Rows:',toronto_data.shape[0])
toronto_data

Number of Rows: 103


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Applied Data Science Capstone - week 3 part 2

##### Get the latitude and the longitude coordinates of each neighborhood.

* Retrieve Coordinates

In [136]:
coord = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_coord = toronto_data.merge(coord, how = 'right',on = 'Postal Code')

In [139]:
toronto_coord.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### Generate Toronto Map

In [140]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(address,'\nLatitude:',latitude,'\nLongitude:',longitude)

Toronto, ON 
Latitude: 43.6534817 
Longitude: -79.3839347


In [142]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_coord['Latitude'], toronto_coord['Longitude'], toronto_coord['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Explore neighbourhoods

* Get the API from Foursquare

In [143]:
CLIENT_ID = 'IKRBHUTQJY3SNPLZZJMANVF4DYA3PPFHRPWYZ0M4UMBIMHWY' # your Foursquare ID
CLIENT_SECRET = '0RZIRXLPPVUAXHLGSWHENBKYBD1POE2DL14FU3FRQMN03BJX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IKRBHUTQJY3SNPLZZJMANVF4DYA3PPFHRPWYZ0M4UMBIMHWY
CLIENT_SECRET:0RZIRXLPPVUAXHLGSWHENBKYBD1POE2DL14FU3FRQMN03BJX


* Select Borough containing 'Toronto'

In [144]:
torData = toronto_coord[toronto_coord['Borough'].str.contains('Toronto')]

In [145]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

* Get 100 Venues of Each Postal Code Area

In [146]:
toronto_venues = getVenues(torData['Neighborhood'], torData['Latitude'], torData['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

#### Cluster neighborhoods

##### Encode Venue Types

In [147]:
toronto_venues
# one hot encoding
toronto_encode = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_encode['Area'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_encode.columns[-1]] + list(toronto_encode.columns[:-1])
toronto_encode = toronto_encode[fixed_columns]

In [149]:
toronto_encode.head()

,Area,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [150]:
toronto_encode.shape

(1604, 233)

##### Get Top 5 Venues for each Neighborhood

In [148]:
#calculate mean venue type for each neighborhood
toronto_sorted = toronto_encode.groupby(by='Area').mean().reset_index()

In [151]:
toronto_sorted.shape

(39, 233)

In [152]:
#Create empty dataframe to be filled
topVen = pd.DataFrame()
#Get top 5 venues for each Area
for count, a in enumerate(toronto_sorted['Area']):
    temp = toronto_sorted[toronto_sorted['Area'] == a]
    temp = temp.drop('Area',axis=1)
    temp.reset_index()
    temp = temp.sort_values(by=count,axis=1,ascending=False)
    topVen = topVen.append(pd.Series(temp.iloc[:,0:5].columns),ignore_index = True)

In [153]:
#Rename Columns & Display first 5 rows
topVen.columns = ['#1 Venue','#2 Venue','#3 Venue','#4 Venue','#5 Venue'] 
topVen.head()

,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant
1,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Italian Restaurant
2,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden
3,Airport Service,Airport Lounge,Plane,Sculpture Garden,Airport Food Court
4,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant


### Cluster Neighborhoods

In [154]:
#fit toronto data to kmeans
toronto_cluster = toronto_sorted.drop(labels='Area',axis=1)
kclusters = 4
kmeans = KMeans(kclusters, random_state=0).fit(toronto_cluster)

In [155]:
topVen.insert(0,'Cluster Labels',kmeans.labels_)

In [156]:
torFinal = torData.sort_values(by='Neighborhood').reset_index(drop=True).join(topVen)
torFinal

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant
1,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Italian Restaurant
2,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558,1,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden
3,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Plane,Sculpture Garden,Airport Food Court
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Italian Restaurant,Diner
6,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gastropub
7,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant
8,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Café,Dessert Shop,Sandwich Place,Coffee Shop
9,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Breakfast Spot,Gym,Park,Sandwich Place,Food & Drink Shop


In [157]:
#initialize map
clusterMap = folium.Map(location=[latitude, longitude], zoom_start=11)

colors = ['blue','red','green','black']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torFinal['Latitude'], torFinal['Longitude'], torFinal['Neighborhood'], torFinal['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(clusterMap)
clusterMap